
<img src="https://gitlab.com/bivl2ab/academico/cursos-uis/ai/ai-2-uis-student/-/raw/master/imgs/Banner_IA2_1.png" style="width:1500px;heigth:100px">

In [ ]:
#@title 1. MONTAR EL DRIVE  (If you are in in colab){ display-mode: "form" }
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Proyecto-IA2')
print(os.getcwd())

In [ ]:
#@title import libs
import os
import cv2
from PIL import Image
import tensorflow as tf
import numpy as np
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from PIL import Image

from tensorflow import keras
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.models import Model

In [ ]:
#Import dataset image from API Kaagle
!pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d ashishmotwani/tomato

In [ ]:
!unzip tomato

In [ ]:
!ls

In [ ]:
SIZE = [225, 255]



model_A = tf.keras.applications.MobileNetV2(SIZE + [3], weights='imagenet', include_top=False)
model_A.trainable = False
model_A.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 225, 255, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 113, 128, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 113, 128, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './train',
        target_size=(100, 100),
        batch_size=100,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        './valid',
        target_size=(100, 100),
        batch_size=100,
        class_mode='categorical')


Found 25851 images belonging to 11 classes.
Found 6683 images belonging to 11 classes.


In [ ]:
from glob import glob
folders = glob("/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/*")
folders

['/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Bacterial_spot',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Early_blight',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Late_blight',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Leaf_Mold',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Septoria_leaf_spot',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Spider_mites Two-spotted_spider_mite',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Target_Spot',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Tomato_Yellow_Leaf_Curl_Virus',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-

In [ ]:
from keras.layers import Flatten
x = Flatten()(model_A.output)
prediction = Dense(len(folders), activation="softmax")(x)
model = Model(inputs=model_A.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 225, 255, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 113, 128, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 113, 128, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [ ]:
learning_rate= 0.1
opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
history = model.fit(train_generator, validation_data=test_generator, epochs=20,verbose=1, batch_size=32)

Epoch 1/20


InvalidArgumentError: ignored

In [ ]:
model = tf.keras.Sequential([model_A, 
                                    tf.keras.layers.GlobalAveragePooling2D(),
                                    tf.keras.layers.Dense(2, activation='softmax')],
                                    name="model")


In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True, monitor='loss')
autosave = tf.keras.callbacks.ModelCheckpoint('model.h5',
                        monitor='loss',
                        mode='min',
                        save_best_only=True,
                        verbose=1)


In [ ]:
results = model.fit(  train_generator, test_generator,
                            epochs=11,
                            verbose=1,
                            batch_size=32,
                            callbacks=[early_stopping, autosave])

Epoch 1/11


InvalidArgumentError: ignored